In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from data import read_data, create_warmup
#from plots import make_all_plots

import scipy

2021-10-21 05:11:43.749355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-21 05:11:43.749386: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [20]:
transition_window =3

warmup_start = '20210428'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

log_dir = './logs/new_warmup'


# Learning rate
learning_rate = 1e-1

In [21]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)

In [22]:
class Comp(Enum):
    A = 0
    M = 1
    #X = 2
    #G = 3
    
class Vax(Enum):
    total = -1
    no = 0
    yes = 1

In [23]:
# get warmup arrays, splitting on vaccination status
warmup_asymp, warmup_mild, warmup_extreme = create_warmup(df, 
                                                          warmup_start, 
                                                          warmup_end,
                                                          0,0,0)

# re-combine
warmup_asymp = warmup_asymp[Vax.no.value] + warmup_asymp[Vax.yes.value]

In [24]:
delta = {}
delta[Vax.total.value] = {}
delta[Vax.total.value]['prior'] = {'a': 1.05, 'b': 20}
# delta is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.1
# I cant figure out the math to do for the scale parameter
delta[Vax.total.value]['posterior_init'] = {'loc': np.log(0.1/(1-0.1)), 'scale':0.1}

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}
# T_serial is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 5.8
# I cant figure out the math to do for the scale parameter
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(5.8),
                                         'scale':0.1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}
# rho is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.76
# I cant figure out the math to do for the scale parameter
rho_M[Vax.total.value]['posterior_init'] = {'loc': np.log(0.76/(1-0.76)),
                                      'scale':0.1}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}
# lambda is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 4.7
# I cant figure out the math to do for the scale parameter
lambda_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(4.7),
                                         'scale':1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}
# nu is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 3.1
# I cant figure out the math to do for the scale parameter
nu_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(3.1),
                                     'scale':1.2}
delta = {}
delta[Vax.total.value] = {}
delta[Vax.total.value]['prior'] = {'a': 1.05, 'b': 20}
# delta is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.1
# I cant figure out the math to do for the scale parameter
delta[Vax.total.value]['posterior_init'] = {'loc': np.log(0.1/(1-0.1)), 'scale':0.1}

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}
# T_serial is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 5.8
# I cant figure out the math to do for the scale parameter
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(5.8),
                                         'scale':0.1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}
# rho is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.76
# I cant figure out the math to do for the scale parameter
rho_M[Vax.total.value]['posterior_init'] = {'loc': np.log(0.76/(1-0.76)),
                                      'scale':0.1}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}
# lambda is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 4.7
# I cant figure out the math to do for the scale parameter
lambda_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(4.7),
                                         'scale':1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}
# nu is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 3.1
# I cant figure out the math to do for the scale parameter
nu_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(3.1),
                                     'scale':1.2}


In [25]:
warmup_A_params = {}
warmup_A_params[Vax.total.value] = {}
warmup_A_params[Vax.total.value]['prior'] = []
warmup_A_params[Vax.total.value]['posterior_init'] = []

for day in range(transition_window):
    warmup_A_params[Vax.total.value]['prior'].append({'loc': warmup_asymp[day],
                                                'scale': warmup_asymp[day]/10})
    # must be positive so reverse softplus the mean
    warmup_A_params[Vax.total.value]['posterior_init'].append({'loc': tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]),dtype=tf.float32),
                                                         'scale': tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]/10),dtype=tf.float32)})

In [26]:
model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                 delta, T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000)
loss = LogPoissonProb() 
optimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate, #momentum=0.98, nesterov=False, name='SGD'
)

In [27]:
x_train = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)
y_test = tf.cast(df.loc[train_start:test_end,'mild'], dtype=tf.float32)


In [28]:
warmup_A_params

{-1: {'prior': [{'loc': 2518.60923786445, 'scale': 251.86092378644497},
   {'loc': 2520.22371776634, 'scale': 252.022371776634},
   {'loc': 2518.626798999444, 'scale': 251.8626798999444}],
  'posterior_init': [{'loc': <tf.Tensor: shape=(), dtype=float32, numpy=2518.6091>,
    'scale': <tf.Tensor: shape=(), dtype=float32, numpy=251.86093>},
   {'loc': <tf.Tensor: shape=(), dtype=float32, numpy=2520.2236>,
    'scale': <tf.Tensor: shape=(), dtype=float32, numpy=252.02237>},
   {'loc': <tf.Tensor: shape=(), dtype=float32, numpy=2518.6267>,
    'scale': <tf.Tensor: shape=(), dtype=float32, numpy=251.86269>}]}}

In [29]:
delta

{-1: {'prior': {'a': 1.05, 'b': 20},
  'posterior_init': {'loc': -2.197224577336219, 'scale': 0.1}}}

In [30]:
A_vals = []
M_vals =[]
for day in range(transition_window):
    A_vals.append(warmup_A_params[-1]['prior'][day]['loc'])

for day in range(len(x_train)):
    yesterday_asymp = A_vals[-1]
    
    today_asymp = yesterday_asymp*0.1*x_train[day]**(1/5.8)
    A_vals.append(today_asymp)
    
    today_M = 0
    pi_M=[]
    for j in range(transition_window):
        
        
        lambda_M_fix = 4.7
        nu_M_fix = 3.1
        poisson_dist = scipy.stats.poisson(lambda_M_fix)
        pi_M_j_ago = poisson_dist.logpmf(j+1)/nu_M_fix
        
        pi_M.append(pi_M_j_ago)
    
    pi_M = scipy.special.softmax(pi_M)
        
    for j in range(transition_window):
        j_ago_asymp = A_vals[day-j-1]
        today_M += j_ago_asymp*0.76*pi_M[j]
        
    M_vals.append(today_M)

In [31]:
model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                 delta, T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000)
loss = LogPoissonProb() 

In [40]:
x_train = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)
y_test = tf.cast(df.loc[train_start:test_end,'mild'], dtype=tf.float32)

with tf.GradientTape() as tape:
    result = model.call(x_train)
    loss_val = loss(y_test, result.stack())
    loss_val += sum(model.losses)

In [41]:
grads = tape.gradient(loss_val, model.trainable_weights)
grads

[<tf.Tensor: shape=(), dtype=float32, numpy=-1.8544123>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0043651485>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0026535513>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0019543308>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-17.15541>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.011383275>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0025674605>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.005413913>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0011504656>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.005970624>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.004317045>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.207721e-05>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.008891908>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.00011962>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.014811867>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.011080822>]

In [42]:
with tf.GradientTape() as tape:
    result = model.call(x_train)
    loss_val = loss(tf.cast(np.array(M_vals),dtype=tf.float32), result.stack())
    loss_val += sum(model.losses)

In [43]:
grads = tape.gradient(loss_val, model.trainable_weights)
grads

[<tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0030840423>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.005278906>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0041279728>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.729411e-05>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0078158025>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-4.0377112e-05>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.013387457>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.011442086>]

In [36]:
model.trainable_weights

[<tf.Variable 'delta_A_loc_-1:0' shape=() dtype=float32, numpy=-2.1972246>,
 <tf.Variable 'delta_A_scale_-1:0' shape=() dtype=float32, numpy=0.1>,
 <tf.Variable 'T_serial_A_loc_-1:0' shape=() dtype=float32, numpy=5.796968>,
 <tf.Variable 'T_serial_A_scale_-1:0' shape=() dtype=float32, numpy=0.1>,
 <tf.Variable 'rho_M_loc_-1:0' shape=() dtype=float32, numpy=1.1526796>,
 <tf.Variable 'rho_M_scale_-1:0' shape=() dtype=float32, numpy=0.1>,
 <tf.Variable 'lambda_M_loc_-1:0' shape=() dtype=float32, numpy=4.690863>,
 <tf.Variable 'lambda_M_scale_-1:0' shape=() dtype=float32, numpy=1.0>,
 <tf.Variable 'nu_M_loc_-1:0' shape=() dtype=float32, numpy=3.0539045>,
 <tf.Variable 'nu_M_scale_-1:0' shape=() dtype=float32, numpy=1.2>,
 <tf.Variable 'warmup_A_loc_-1:0' shape=() dtype=float32, numpy=2518.6091>,
 <tf.Variable 'warmup_A_scale_-1:0' shape=() dtype=float32, numpy=251.86093>,
 <tf.Variable 'warmup_A_loc_-1:0' shape=() dtype=float32, numpy=2520.2236>,
 <tf.Variable 'warmup_A_scale_-1:0' shape=(

In [44]:
M_vals-result.stack()

InvalidArgumentError: Incompatible shapes: [123] vs. [123,1000] [Op:Sub]

In [39]:
model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
model.fit(x=x_train,
         epochs=2000, batch_size=0,
        callbacks=[])

Epoch 1/2000
4/4 [==============================] - 7s 2s/step - loss: 79.7895
Epoch 2/2000
4/4 [==============================] - 7s 2s/step - loss: 143.9761
Epoch 3/2000
4/4 [==============================] - 7s 2s/step - loss: 203.6338
Epoch 4/2000
4/4 [==============================] - 7s 2s/step - loss: 260.5018
Epoch 5/2000
4/4 [==============================] - 7s 2s/step - loss: 315.9380
Epoch 6/2000
4/4 [==============================] - 7s 2s/step - loss: 370.9204
Epoch 7/2000
4/4 [==============================] - 7s 2s/step - loss: 425.4656
Epoch 8/2000
4/4 [==============================] - 7s 2s/step - loss: 479.7756
Epoch 9/2000
2/4 [==============>...............] - ETA: 3s - loss: 520.4515WARNING:tensorflow:Gradients do not exist for variables ['warmup_A_loc_-1:0', 'warmup_A_scale_-1:0', 'warmup_A_loc_-1:0', 'warmup_A_scale_-1:0'] when minimizing the loss.


4/4 [==============================] - 7s 2s/step - loss: 533.8607
Epoch 10/2000
4/4 [==============================] - 7s 2s/step - loss: 587.2517
Epoch 11/2000
4/4 [==============================] - 7s 2s/step - loss: 641.0140
Epoch 12/2000
4/4 [==============================] - 7s 2s/step - loss: 694.6429
Epoch 13/2000
4/4 [==============================] - 7s 2s/step - loss: 748.3470
Epoch 14/2000
4/4 [==============================] - 7s 2s/step - loss: 802.0098
Epoch 15/2000
4/4 [==============================] - 7s 2s/step - loss: 855.7876
Epoch 16/2000
4/4 [==============================] - 7s 2s/step - loss: 909.4814
Epoch 17/2000
4/4 [==============================] - 7s 2s/step - loss: 963.0253
Epoch 18/2000
1/4 [======>.......................] - ETA: 5s - loss: 996.2737WARNING:tensorflow:Gradients do not exist for variables ['warmup_A_loc_-1:0', 'warmup_A_scale_-1:0', 'warmup_A_loc_-1:0', 'warmup_A_scale_-1:0'] when minimizing the loss.


4/4 [==============================] - 7s 2s/step - loss: 1016.4074
Epoch 19/2000
4/4 [==============================] - 7s 2s/step - loss: 1069.7600
Epoch 20/2000
4/4 [==============================] - 7s 2s/step - loss: 1122.9342
Epoch 21/2000
4/4 [==============================] - 7s 2s/step - loss: 1176.2189
Epoch 22/2000
4/4 [==============================] - 7s 2s/step - loss: 1229.2382
Epoch 23/2000
4/4 [==============================] - 7s 2s/step - loss: 1282.4529
Epoch 24/2000
4/4 [==============================] - 7s 2s/step - loss: 1336.0553
Epoch 25/2000
4/4 [==============================] - 7s 2s/step - loss: 1389.5417
Epoch 26/2000
3/4 [=====================>........] - ETA: 1s - loss: 1435.9580WARNING:tensorflow:Gradients do not exist for variables ['warmup_A_loc_-1:0', 'warmup_A_scale_-1:0', 'warmup_A_loc_-1:0', 'warmup_A_scale_-1:0'] when minimizing the loss.


4/4 [==============================] - 7s 2s/step - loss: 1442.5328
Epoch 27/2000
4/4 [==============================] - 7s 2s/step - loss: 1495.6031
Epoch 28/2000
4/4 [==============================] - 7s 2s/step - loss: 1549.0386
Epoch 29/2000
4/4 [==============================] - 7s 2s/step - loss: 1602.2185
Epoch 30/2000
4/4 [==============================] - 7s 2s/step - loss: 1655.3661
Epoch 31/2000
4/4 [==============================] - 7s 2s/step - loss: 1708.4144
Epoch 32/2000
4/4 [==============================] - 7s 2s/step - loss: 1761.3210
Epoch 33/2000
4/4 [==============================] - 7s 2s/step - loss: 1814.5020
Epoch 34/2000
4/4 [==============================] - 7s 2s/step - loss: 1868.0568
Epoch 35/2000
1/4 [======>.......................] - ETA: 5s - loss: 1901.2886WARNING:tensorflow:Gradients do not exist for variables ['warmup_A_loc_-1:0', 'warmup_A_scale_-1:0', 'warmup_A_loc_-1:0', 'warmup_A_scale_-1:0'] when minimizing the loss.


4/4 [==============================] - 7s 2s/step - loss: 1921.0969
Epoch 36/2000
4/4 [==============================] - 7s 2s/step - loss: 1974.1609
Epoch 37/2000
4/4 [==============================] - 7s 2s/step - loss: 2027.1195
Epoch 38/2000
4/4 [==============================] - 7s 2s/step - loss: 2080.1062
Epoch 39/2000
4/4 [==============================] - 8s 2s/step - loss: 2133.1333
Epoch 40/2000
4/4 [==============================] - 7s 2s/step - loss: 2186.1902
Epoch 41/2000
4/4 [==============================] - 7s 2s/step - loss: 2238.7434
Epoch 42/2000
4/4 [==============================] - 7s 2s/step - loss: 2291.6404
Epoch 43/2000
3/4 [=====================>........] - ETA: 1s - loss: 2337.9331WARNING:tensorflow:Gradients do not exist for variables ['warmup_A_loc_-1:0', 'warmup_A_scale_-1:0', 'warmup_A_loc_-1:0', 'warmup_A_scale_-1:0'] when minimizing the loss.


4/4 [==============================] - 7s 2s/step - loss: 2344.6262
Epoch 44/2000
4/4 [==============================] - 7s 2s/step - loss: 2397.9365
Epoch 45/2000
4/4 [==============================] - 7s 2s/step - loss: 2450.7720
Epoch 46/2000
4/4 [==============================] - 7s 2s/step - loss: 2503.2949
Epoch 47/2000


KeyboardInterrupt: 

In [72]:
pi_M

array([0.26041826, 0.34306052, 0.39652121])

In [73]:
r_t

NameError: name 'r_t' is not defined

In [74]:
x_train

<tf.Tensor: shape=(123,), dtype=float32, numpy=
array([0.9893859 , 0.9806603 , 0.9708389 , 0.9601291 , 0.9487467 ,
       0.93690044, 0.9247909 , 0.9126141 , 0.90057087, 0.8888542 ,
       0.87764704, 0.867123  , 0.85742813, 0.84862584, 0.840759  ,
       0.83387077, 0.82800525, 0.8231926 , 0.819385  , 0.8165125 ,
       0.8145094 , 0.8133129 , 0.8128637 , 0.81310666, 0.8139891 ,
       0.8154599 , 0.8174688 , 0.81997126, 0.8229714 , 0.82650083,
       0.83059275, 0.83528215, 0.84060645, 0.8466147 , 0.8533651 ,
       0.8609193 , 0.86934257, 0.8787029 , 0.8890385 , 0.90036035,
       0.9126806 , 0.9260135 , 0.9403739 , 0.95573395, 0.9720139 ,
       0.98912555, 1.0069746 , 1.0254581 , 1.0444374 , 1.0637217 ,
       1.0831019 , 1.1023542 , 1.121239  , 1.1395118 , 1.1569333 ,
       1.173255  , 1.1882203 , 1.2015666 , 1.2130756 , 1.2226428 ,
       1.2301873 , 1.2356372 , 1.2389311 , 1.2400631 , 1.2391775 ,
       1.2364545 , 1.2320814 , 1.2262504 , 1.2191668 , 1.2110765 ,
       1.20223

In [75]:
M_vals

[<tf.Tensor: shape=(), dtype=float32, numpy=1465.9128>,
 <tf.Tensor: shape=(), dtype=float32, numpy=580.7713>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1156.2155>,
 1914.5674333244156,
 <tf.Tensor: shape=(), dtype=float32, numpy=1465.9128>,
 <tf.Tensor: shape=(), dtype=float32, numpy=829.509>,
 <tf.Tensor: shape=(), dtype=float32, numpy=82.78683>,
 <tf.Tensor: shape=(), dtype=float32, numpy=8.249547>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.82061327>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.08147247>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.008072095>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00079802796>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.871807e-05>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.747087e-06>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.6069267e-07>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.452509e-08>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.2853448e-09>,
 <tf.Tensor: shape=(), dtype=float32, numpy=7.107225e-10>,
 <tf

In [76]:
A_vals

[2518.60923786445,
 2520.22371776634,
 2518.626798999444,
 <tf.Tensor: shape=(), dtype=float32, numpy=251.39973>,
 <tf.Tensor: shape=(), dtype=float32, numpy=25.05547>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.492795>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.2475369>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.024530157>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.002425604>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00023931247>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.355691e-05>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.3135378e-06>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.2670145e-07>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.216572e-08>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.162749e-09>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.1061462e-10>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.047379e-11>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.987058e-12>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.925781e-13>,
 <tf.Tenso

In [46]:
result.stack()

<tf.Tensor: shape=(123, 1000), dtype=float32, numpy=
array([[6.0227575, 5.7063975, 5.2482114, ..., 5.664287 , 5.727786 ,
        4.844395 ],
       [4.058896 , 3.8579164, 3.6071632, ..., 4.1506524, 3.8499112,
        3.2580187],
       [2.1488903, 2.1440263, 1.9318486, ..., 2.0882523, 2.0435781,
        1.4867159],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)>

In [49]:
model.trainable_weights

[<tf.Variable 'delta_A_loc_-1:0' shape=() dtype=float32, numpy=-2.9499655>,
 <tf.Variable 'delta_A_scale_-1:0' shape=() dtype=float32, numpy=-2.772707>,
 <tf.Variable 'T_serial_A_loc_-1:0' shape=() dtype=float32, numpy=5.7982707>,
 <tf.Variable 'T_serial_A_scale_-1:0' shape=() dtype=float32, numpy=-3.087353>,
 <tf.Variable 'rho_M_loc_-1:0' shape=() dtype=float32, numpy=1.1270646>,
 <tf.Variable 'rho_M_scale_-1:0' shape=() dtype=float32, numpy=-3.1242275>,
 <tf.Variable 'lambda_M_loc_-1:0' shape=() dtype=float32, numpy=4.6978197>,
 <tf.Variable 'lambda_M_scale_-1:0' shape=() dtype=float32, numpy=-2.5303206>,
 <tf.Variable 'nu_M_loc_-1:0' shape=() dtype=float32, numpy=3.1201112>,
 <tf.Variable 'nu_M_scale_-1:0' shape=() dtype=float32, numpy=-2.4528675>,
 <tf.Variable 'warmup_A_loc_-1:0' shape=() dtype=float32, numpy=2518.6091>,
 <tf.Variable 'warmup_A_scale_-1:0' shape=() dtype=float32, numpy=251.86093>,
 <tf.Variable 'warmup_A_loc_-1:0' shape=() dtype=float32, numpy=2520.2236>,
 <tf.Var

In [48]:
np.log(-2.9/(1-2.9))

0.4228568508200336